In [21]:
import numpy as np
import scipy
from sklearn.model_selection import train_test_split
import pandas as pd

from implicit.als import AlternatingLeastSquares
from implicit.evaluation import mean_average_precision_at_k

Загрузим наш датасет, добавим имена столбцам, затем отсортируем записи по дате и преобразуем целевую переменную в бинарный вид: условимся, что фильмы, которым пользователь поставил оценку выше 2, считаются понравившимися ему (класс 1), в противном случае — не понравившимися (класс 0):

In [22]:
ratings = pd.read_csv("./data/ml-100k/u.data", sep="\t", header=None)
ratings.columns = ['user_id', 'item_id', 'rating', 'timestamp']
ratings.sort_values('timestamp', inplace=True)
ratings['score'] = (ratings['rating'] > 2).apply(int)

In [23]:
display(
    ratings.head(),
    ratings.tail()
)

,user_id,item_id,rating,timestamp,score
214,259,255,4,874724710,1
83965,259,286,4,874724727,1
43027,259,298,4,874724754,1
21396,259,185,4,874724781,1
82655,259,173,4,874724843,1


,user_id,item_id,rating,timestamp,score
46773,729,689,4,893286638,1
73008,729,313,3,893286638,1
46574,729,328,3,893286638,1
64312,729,748,4,893286638,1
79208,729,272,4,893286638,1


Затем разделим нашу выборку на тренировочную и тестовую в соотношении 80/20 без перемешивания. В тренировочную выборку попадут оценки пользователей за первые 80 % периода проведения наблюдений, а в тестовую — оставшиеся 20 %:

In [24]:
train, test = train_test_split(ratings, test_size=0.2, shuffle=False)

Чтобы обучить ALS-модель на предоставленных данных, нужно создать user-item таблицу для тренировочной и тестовой выборки. В этой таблице по строкам должны быть отложены идентификаторы всех уникальных пользователей, которые у нас есть, а по столбцам — все уникальные фильмы. То есть мы должны получить две матрицы размерности 943 x 1682. На пересечении строк и столбцов этих матриц должны быть числа, характеризующие наличие положительных и отрицательных оценок пользователей. 

Проблема заключается в том, что в тренировочную и тестовую выборку могли попасть различные пользователи и различные товары. Посмотрим на это, создав сводные таблицы для тренировочной и тестовой выборок. 

In [25]:
train_pivot = pd.pivot_table(train, index="user_id", columns="item_id", values="score")
test_pivot = pd.pivot_table(test, index="user_id", columns="item_id", values="score")

display(train_pivot.head(), test_pivot.tail())

print(train_pivot.shape)
print(test_pivot.shape)

item_id,1,2,3,4,5,6,7,8,9,10,...,1660,1662,1664,1671,1672,1675,1676,1677,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,NaN,1.0,1.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1.0,NaN,NaN,NaN,NaN,NaN,0.0,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


item_id,1,2,3,4,5,6,7,8,9,10,...,1667,1668,1669,1670,1672,1673,1674,1678,1679,1680
user_id,,,,,,,,,,,,,,,,,,,,,
932,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
934,0.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
938,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
942,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


(751, 1616)
(301, 1448)


Видим, что в тренировочную user-item таблицу попали оценки от 751 пользователей для 1616 товаров, а в тестовую — от 301 пользователей для 1448 товаров. 

Теперь создадим сводную таблицу из таблицы rating, заполнив её ячейки нулями. Получим матрицу размером 943 x 1682. Для тех фильмов, которым пользователь выставил оценку значения, будут равны 0, для остальных — пропуску. 

In [26]:
shell = pd.pivot_table(
    ratings, 
    index="user_id", 
    columns="item_id", 
    values="score", 
    aggfunc=lambda x: 0
)
shell.head()

item_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Чтобы получить тренировочную и тестовую user-item таблицы, нам осталось только сложить таблицу shell с соответствующими таблицами train_pivot и test_pivot.

*Примечание*. Сложение DataFrame производится на пересечении соответствующих индексов строк и столбцов. Элементы, которые не попали в пересечение помечаются как пропуски. 

Чтобы корректно обрабатывать пропущенные значения, мы трансформируем 1 в 2, а 0 — в 1. Сами пропуски заполняем нулями. В результате у нас получатся две таблицы размером 943 x 1682, в которых на пересечении пользователя и фильма стоит:

0 — если пользователь не оценил данный фильм;

1 — если пользователь оценил фильм отрицательно;

2 — если пользователь оценил фильм положительно.

In [27]:
train_pivot = shell + train_pivot
test_pivot = shell + test_pivot

train_pivot = (train_pivot + 1).fillna(0)
test_pivot = (test_pivot + 1).fillna(0)
print(train_pivot.shape)
print(test_pivot.shape)
## (943, 1682)
## (943, 1682)

train_pivot.head()

(943, 1682)
(943, 1682)


item_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,2.0,2.0,2.0,2.0,0.0,2.0,2.0,1.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Финальный шаг предобработки: модели из библиотеки implicit требуют, чтобы user-item матрицы были представлены в виде разреженных матриц. Для получения разреженной матрицы используется функция csr_matrix() из модуля sparse библиотеки scipy:

In [28]:
train_pivot_sparse = scipy.sparse.csr_matrix(train_pivot.values)
test_pivot_sparse = scipy.sparse.csr_matrix(test_pivot.values)

Теперь, когда созданы тренировочная и тестовая user-item таблицы, мы готовы перейти к этапу построения модели. Обучим ALS-модель с 10-ю факторами, параметр random_state установим в значение 42.

In [29]:
model = AlternatingLeastSquares(factors=10, random_state=42)
model.fit(train_pivot_sparse)

  0%|          | 0/15 [00:00<?, ?it/s]

Чтобы сформировать рекомендации для конкретного пользователя, можно воспользоваться методом recommend(). 

Метод принимает идентификатор пользователя, для которого нужно сформировать рекомендации и его вектор оценок объектам (фильмам). Вектор оценок достаём из разреженной матрицы, на которой обучались. 

Метод возвращает индексы товаров (фильмов), которые, по мнению ALS, являются наиболее вероятными для покупки (просмотра) данным пользователем, а также веса этих товаров. 

**Важный момент**: индексы товаров не равны их идентификаторам! То есть метод recommend() возвращает номера столбцов из user-item таблицы, а не сами идентификаторы товаров! 

Номера столбцов начинаются от 0, а вот идентификаторы фильмов в нашем примере выражаются числами и начинаются от 1. Поэтому необходимо сначала выделить все уникальные идентификаторы товаров в отдельный вектор, а уже затем обращаться к нему по полученным индексам.

In [30]:
unique_items = np.array(train_pivot.columns)
user_id = 14
recomendations_ids, scores = model.recommend(user_id, train_pivot_sparse[user_id])
recomendations = unique_items[recomendations_ids]
print("Recomendations ids: {}".format(recomendations_ids))
print("Recomendations for user {}: {}".format(user_id, recomendations))

Recomendations ids: [293 116 244 275 287  99 283 312 596 150]
Recomendations for user 14: [294 117 245 276 288 100 284 313 597 151]


Итак, для пользователя с идентификатором 14 наша рекомендательная система рекомендовала фильмы под идентификаторами [294 117 100 276 245 288 284 151 126 597]. Чтобы понять, что это за фильмы, можно обратиться к таблице movies, которую мы рассматривали ранее в модуле. 

Теперь определим качество модели на всей тестовой выборке, рассчитав precision для топ 10-рекомендуемых фильмов с помощью функции mean_average_precision_at_k() из библиотеки implicit:

In [31]:
map_at10 = mean_average_precision_at_k(model, train_pivot_sparse, test_pivot_sparse, K=10)
print('Mean Average Precision at 10: {:.3f}'.format(map_at10))


  0%|          | 0/301 [00:00<?, ?it/s]

Mean Average Precision at 10: 0.084


----
Давайте попробуем улучшить качество рекомендаций, воспользовавшись факторизационными машинами. 

Установим библиотеку lightfm, в которой содержатся методы матричной факторизации:

In [32]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k

Обучим факторизационные машины с 10-ю факторами, в качестве функции потерь используем logloss, параметр random_state установим в значение 42. Обучение будем производить на 30 итерациях (эпохах):

In [34]:
model = LightFM(no_components=10, loss='logistic', random_state=42)
model.fit(train_pivot_sparse, epochs=30)

Процесс построения рекомендаций для пользователей у моделей из модуля lightfm сильно отличается от того же процесса у моделей из implicit. 

Предсказание осуществляется с помощью метода predict(), который принимает на вход идентификатор пользователя, а также индексы всех объектов (фильмов); индексы обязательно начинаются от 0. Метод возвращает веса для каждого объекта, причём веса объектов отрицательные. 

Чтобы получить сами рекомендации, необходимо умножить эти веса на -1 и отсортировать их индексы по возрастанию веса. Нам нужны будут только индексы 10 фильмов с наибольшим по модулю весом. Обратившись по полученным индексам к списку идентификаторов фильмов, мы получим рекомендации для конкретного пользователя:

In [35]:
item_ids = np.arange(0, train_pivot_sparse.shape[1])
list_pred = model.predict(user_id, item_ids)
recomendations_ids = np.argsort(-list_pred)[:10]
recomendations = unique_items[recomendations_ids]
print('Recomendations for user {}: {}'.format(user_id, recomendations))

## Recomendations for user 14: [ 50 294 258 100 181 288 286   1 300 121]

Recomendations for user 14: [ 50 294 258 100 181 288 286   1 300 121]


Наконец, рассчитаем средний precision для топ 10 рекомендуемых фильмов по всей тестовой выборке. Для этого можно воспользоваться функцией precision_at_k() из библиотеки lightfm. Результатом будет значение precision@k для каждого из пользователей. 

In [36]:
map_at10 = precision_at_k(model, test_pivot_sparse, k=10).mean()
print('Mean Average Precision at 10: {:.2f}'.format(map_at10))
## Mean Average Precision at 10: 0.32

Mean Average Precision at 10: 0.32
